# Prerequisites

In [1]:
%%capture
pip install requests docling transformers ipywidgets

In [2]:
%%capture
pip install llama-index-core llama-index-readers-docling llama-index-node-parser-docling llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-vector-stores-milvus llama-index-readers-file python-dotenv;

In [3]:
## Docling and chunker for text parsing
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
## embeddings generator
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Docling ingestion

## Define global variable

In [4]:
## Source for the documents for docling parsing of chunks
source_file = "http://minio-ui-ic-shared-minio.apps.ocp.sandbox2941.opentlc.com/api/v1/download-shared-object/aHR0cDovLzEyNy4wLjAuMTo5MDAwL3BkZnMvUmVkX0hhdF9PcGVuU2hpZnRfQUlfU2VsZi1NYW5hZ2VkLTIuMTYtR2V0dGluZ19zdGFydGVkX3dpdGhfUmVkX0hhdF9PcGVuU2hpZnRfQUlfU2VsZi1NYW5hZ2VkLWVuLVVTLnBkZj9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUVNWURIR0pFNDBQQzczVElMWVBFJTJGMjAyNTAxMTQlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwMTE0VDE0MTIxNFomWC1BbXotRXhwaXJlcz00MzIwMCZYLUFtei1TZWN1cml0eS1Ub2tlbj1leUpoYkdjaU9pSklVelV4TWlJc0luUjVjQ0k2SWtwWFZDSjkuZXlKaFkyTmxjM05MWlhraU9pSkZUVmxFU0VkS1JUUXdVRU0zTTFSSlRGbFFSU0lzSW1WNGNDSTZNVGN6Tmprd056QTNPQ3dpY0dGeVpXNTBJam9pYldsdWFXOGlmUS53c2NjZ3dCSVdoYl92UFBYMmlyZUZxVHh4MzZyN18xX3JKcXZLUUcxWVhJXzh1TXBKQUxzb0xkRWU3VXBROE5lWHFtdzNRX1ppU1duUDdXSzJ0SlFYdyZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QmdmVyc2lvbklkPW51bGwmWC1BbXotU2lnbmF0dXJlPWY1MDBmNDc4MThhYzE3YjU4NmQ5YmU4ZTkyMzI3OTQ0ZDlhNmQ5YWFiNDU1ZjI4ZmQzM2I3ODVmYTc0NjliZDA"  # document per local path or URL

## 0) Ingest documentation - Test

In [5]:
## Define Converter
converter = DocumentConverter()

## Parse document from source and store in variable "document"
document = converter.convert(source_file)

## Print to check everything is working
print(document.document.export_to_markdown())


<!-- image -->

## Red Hat OpenShift AI Self-Managed 2.16

## Getting started with Red Hat OpenShift AI Self-Managed

Learn how to work in an OpenShift AI environment

## Red Hat OpenShift AI Self-Managed 2.16 Getting started with Red Hat OpenShift AI Self-Managed

Learn how to work in an OpenShift AI environment

## Legal Notice

Copyright © 2024 Red Hat, Inc.

The text of and illustrations in this document are licensed by Red Hat under a Creative Commons Attribution-Share Alike 3.0 Unported license ("CC-BY-SA"). An explanation of CC-BY-SA is available at

http://creativecommons.org/licenses/by-sa/3.0/

- . In accordance with CC-BY-SA, if you distribute this document or an adaptation of it, you must provide the URL for the original version.

Red Hat, as the licensor of this document, waives the right to enforce, and agrees not to assert, Section 4d of CC-BY-SA to the fullest extent permitted by applicable law.

Red Hat, Red Hat Enterprise Linux, the Shadowman logo, the Red Hat logo, J

## 1) Chunking the downloaded document using Docling

In [6]:
## Parse document from source and store in variable "document"
converted_source_file = DocumentConverter().convert(source_file)
document = converted_source_file.document

## Create chubker and chuck document
chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")  # set tokenizer as needed
chunk_iter = chunker.chunk(document)

## Create chunk_list with the parts of the document
chunk_list = list(chunk_iter)

#for i,chunk in enumerate(chunk_iter):
    #print(i)
    #print(chunk)

## Print to check everything is working
# print(chunk_list[0])
print(chunk_list[0].text)

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


Learn how to work in an OpenShift AI environment
Last Updated: 2024-12-11


## 2) Create embeddings

In [7]:
## Defined embedding model and import it
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Define Empty Vector Array
vectors = []
## Populate Vector Array
for i, chunk in enumerate(chunk_list):
    vectors.append({
        "id": i, 
        "vector": embedding_model.get_text_embedding(chunk.text) , 
        "text": chunk.text
    })

## Print to check everything is working
print(f'Length of the Vector: {len(vectors)}' )

Length of the Vector: 47


## Upload embeddings to Milvus

In [8]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

## Connect to Milvus (adjust host/port as needed)
connections.connect("default", 
    host="vectordb-milvus.milvus.svc.cluster.local", 
    port="19530",
    token="root:Milvus"
)

## Define a collection schema (adjust dimensions based on your embedding size)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=len(vectors[0]["vector"])),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65000),
    ## TODO: Add version
    #    FieldSchema(name="version", dtype=DataType.VARCHAR, max_length=1024),
]
schema = CollectionSchema(fields, description="RAG embeddings collection")

## TODO: search
# res = client.search(
#     collection_name="my_collection",
#     data=[query_vector],
#     limit=5,
#     # highlight-start
#     filter='color like "red%" and likes > 50',
#     output_fields=["color", "likes"]
#     # highlight-end
# )


## Create or load a collection
collection_name = "rag_embeddings"
collection = Collection(name=collection_name, schema=schema)

## TODO: Create an index for faster similarity search
index_params = {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": len(vectors[0]["vector"])}}
collection.create_index(field_name="vector", index_params=index_params)

## Load the collection for querying
collection.load()

In [9]:
## Insert vectors into Milvus
insert_output = collection.insert(vectors);

## Print to check everything is working
print(insert_output)

## Load the collection for querying
collection.load()

(insert count: 47, delete count: 0, upsert count: 0, timestamp: 455311954974343175, success count: 47, err count: 0


## 3) Query the Database to obtain RAG parsed information

In [10]:
## TODO: 

## 4) Query Mistral Using RAG information

In [11]:
prompt =f"""
You are a commentator. Your task is to write a report on an essay.
When presented with the essay, come up with interesting questions to ask, and answer each question.
Afterward, combine all the information and write a report in the markdown format.

# Essay:
{markdown_dummy_2}

# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes presented in the essay.

## Interesting Questions:
Generate three distinct and thought-provoking questions that can be asked about the content of the essay. For each question:
- After "Q: ", describe the problem
- After "A: ", provide a detailed explanation of the problem addressed in the question.
- Enclose the ultimate answer in <>.

## Write a report
Using the essay summary and the answers to the interesting questions, create a comprehensive report in Markdown format.
"""



NameError: name 'markdown_dummy_2' is not defined